# Convert data to arrays and encode categorical data 

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler

In [2]:
train = pd.read_csv('../data/train.csv')#, na_values=-1)
test = pd.read_csv('../data/test.csv')#, na_values=-1)

In [3]:
#test = train.iloc[:10000].copy()

In [4]:
cat = [ i for i in train.columns if "cat" in i ]
train[cat] = train[cat].apply( lambda d: d.replace(-1, d.max()+1), axis=0 )
test[cat] = test[cat].apply( lambda d: d.replace(-1, d.max()+1), axis=0 )

test_ids = test['id']
train.drop(['ps_car_11_cat','id'] ,axis=1,inplace=True)
test.drop(['ps_car_11_cat','id'] ,axis=1,inplace=True)

train = train.sample(frac=1).reset_index(drop=True)
#test  = test.sample(frac=1).reset_index(drop=True)


In [5]:
encoder = OneHotEncoder(sparse=False)
train_arr = train['target'].values.reshape(-1,1)
test_arr = np.empty([test.shape[0],0])
scaler = StandardScaler()

for feature in train.columns[1:] :
    
    if "cat" in feature :
        print "categorical column :\t", feature, " - ", train[feature].value_counts().shape[0]
        
        _ = encoder.fit(np.concatenate((train[feature].values,test[feature].values)).reshape(-1,1))
        train_arr = np.append(train_arr, encoder.transform(train[feature].values.reshape(-1,1)), axis=1)
        test_arr = np.append(test_arr, encoder.transform(test[feature].values.reshape(-1,1)), axis=1)
    
    elif "bin" in feature :
        train_arr = np.append(train_arr, train[feature].values.reshape(-1,1), axis=1)
        test_arr = np.append(test_arr, test[feature].values.reshape(-1,1), axis=1)
    
    else :
        #_ = scaler.fit(train[feature].values.reshape(-1,1))
        _ = scaler.fit(np.concatenate((train[feature].values,test[feature].values)).reshape(-1,1))
        train_arr = np.append(train_arr, scaler.transform(train[feature].values.reshape(-1,1)), axis=1)
        test_arr = np.append(test_arr, scaler.transform(test[feature].values.reshape(-1,1)), axis=1)
        
print "\n\nnew train shape : ", train_arr.shape
print "\n\nnew test shape : ", test_arr.shape
# the column [:,0] is for labels.

/usr/lib64/python2.7/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, _DataConversionWarning)


categorical column :	ps_ind_02_cat  -  5
categorical column :	ps_ind_04_cat  -  3
categorical column :	ps_ind_05_cat  -  8
categorical column :	ps_car_01_cat  -  13
categorical column :	ps_car_02_cat  -  3
categorical column :	ps_car_03_cat  -  3
categorical column :	ps_car_04_cat  -  10
categorical column :	ps_car_05_cat  -  3
categorical column :	ps_car_06_cat  -  18
categorical column :	ps_car_07_cat  -  3
categorical column :	ps_car_08_cat  -  2
categorical column :	ps_car_09_cat  -  6
categorical column :	ps_car_10_cat  -  3


new train shape :  (595212, 124)


new test shape :  (892816, 123)


In [6]:
# save to disk
np.save('../nn/data/train_scaled.npy',train_arr)
np.save('../nn/data/test_scaled.npy',test_arr)
#np.save('../nn/data/test_ids.npy', test_ids)

In [7]:
# save samples to disk
np.save('../nn/data/train_scaled_sample.npy',train_arr[0:100000,:])
np.save('../nn/data/test_scaled_sample.npy', test_arr[0:100000,:])

In [8]:
train_arr.shape

(595212, 124)

--------------